In [1]:
import black
import jupyter_black

jupyter_black.load(
    line_length=79,
    verbosity="DEBUG",
    target_version=black.TargetVersion.PY310,
)

DEBUG:jupyter_black:config: {'line_length': 79, 'target_versions': {<TargetVersion.PY310: 10>}}


In [2]:
# imports
import random
import hashlib as h
import time
import numpy as np

hash_data_type = np.dtype("S4")
num_bytes = 4
zeros = 0

list_of_password_extractions = []
list_of_colliding_hashes = []

In [3]:
"""Helper function to visualise passwords as hex strings"""


def bytes_to_string(bytes_data):
    intRepresentation = int.from_bytes(bytes_data, "big")
    return "0x{0:x}".format(intRepresentation)

In [4]:
"""Write the generated results to a .txt file"""


def writeResults(p):
    f = open("HellmanDataTemp.txt", "a")
    for i in range(0, 13):
        f.write(str(p[i]) + "--")
    f.write("\n")
    f.close()

In [5]:
"""h(x) - apply only hash function"""


def apply_hash(current):
    current = h.sha1(current).digest()[:num_bytes]
    return current

In [6]:
"""Apply function - hash and then XOR."""


def apply_function(i, current):
    current = apply_hash(current)
    current = bytes(x ^ y for x, y in zip(i, current))[:num_bytes]
    return current

In [7]:
"""Generate random password of a given size"""


def get_random_password(size):
    new = random.getrandbits(size)
    password = new.to_bytes(num_bytes, "big")
    hashed_value = apply_function(zeros, password)
    return (password, hashed_value)

In [8]:
"""Generate a list of random hashes to attempt to invert"""


def generate_passwords(num, num_bits):
    my_passwords = list()
    for i in range(num):
        my_passwords.append(get_random_password(num_bits))
    return my_passwords

In [9]:
"""Generate a single random start point and returns it"""


def get_start_point(input_size):
    new = random.getrandbits(input_size)
    sp = new.to_bytes(num_bytes, "big")
    return sp

In [10]:
"""Repeatedly call the get_start_point function in order to generate all requiered start points"""


def gen_start_points(num_tables, num_chains, input_size):
    global num_bytes
    num_bytes = input_size // 8

    start_points = [[0 for i in range(num_chains)] for j in range(num_tables)]
    start_points = np.array(start_points, dtype=hash_data_type)

    for i in range(0, num_tables):
        for j in range(0, num_chains):
            start_points[i, j] = get_start_point(input_size)

    return start_points

In [11]:
"""Calculate the end point for a given start point in a given table"""


def get_end_point(start_point, table_num, chain_length):
    current = start_point
    for i in range(0, chain_length):
        current = apply_function(table_num.to_bytes(num_bytes, "big"), current)
    return current

In [12]:
"""Generate end point method"""


def gen_end_points(start_points, chain_length):

    num_tables = len(start_points)
    num_chains = len(start_points[0])

    end_points = [[0 for i in range(num_chains)] for j in range(num_tables)]
    end_points = np.array(end_points, dtype=hash_data_type)

    # iterate through tables
    for c in range(0, num_tables):
        # itereate through start points
        for i in range(0, num_chains):
            end_points[c, i] = get_end_point(
                start_points[c, i], c, chain_length
            )

    return end_points

In [13]:
"""Method to recalculate chain up to a specified stop point"""


def find_chain_entry(chain_number, stop_point, start_points, table_number):
    current = start_points[table_number, chain_number]
    c = table_number.to_bytes(num_bytes, "big")
    for i in range(0, stop_point):
        current = apply_function(c, current)
    return current

In [14]:
"""Inverse hash method"""


def search_chains(tp, y, start_points, end_points, chain_length):

    true_password = tp
    hash_of_password = y
    num_tables = len(start_points)
    num_chains = len(start_points[0])
    false_alarms = 0
    gen_hashes = 0
    false_alarm_hashes = 0
    success_hashes = 0

    global list_of_password_extractions
    global list_of_colliding_hashes

    for x in range(0, num_tables):

        hash_of_password = y

        c = x.to_bytes(4, "big")

        # applying XOR
        hash_of_password = bytes(a ^ b for a, b in zip(c, hash_of_password))[
            :4
        ]

        # generate list
        chain_for_y = list()
        chain_for_y.append(hash_of_password)
        hash_copy = hash_of_password

        for z in range(0, chain_length):
            hash_copy = apply_function(c, hash_copy)
            chain_for_y.append(hash_copy)

        # perform hash if no matching end point is found up until the length of the chain
        for j in range(0, chain_length):
            # check each chains endpoints
            for i in range(0, num_chains):

                current_end_point = end_points[x, i]

                if (current_end_point == hash_of_password) and (
                    chain_for_y.count(current_end_point) != 0
                ):
                    # if (current_end_point == hash_of_password):

                    # the password / input is the previous member of the chain, so recompute from corresponding start point
                    password_guess = find_chain_entry(
                        chain_number=i,
                        stop_point=chain_length - j - 1,
                        start_points=start_points,
                        table_number=x,
                    )
                    temp_hash_count = chain_length - j - 1

                    if password_guess == true_password:
                        success_hashes = temp_hash_count
                        list_of_password_extractions.append(
                            [
                                bytes_to_string(tp),
                                bytes_to_string(y),
                                bytes_to_string(password_guess),
                            ]
                        )
                        return (
                            True,
                            false_alarms,
                            gen_hashes,
                            false_alarm_hashes,
                            success_hashes,
                        )

                    if apply_hash(password_guess) == apply_hash(true_password):
                        success_hashes = temp_hash_count
                        list_of_colliding_hashes.append(
                            [
                                bytes_to_string(tp),
                                bytes_to_string(y),
                                bytes_to_string(password_guess),
                            ]
                        )
                        return (
                            True,
                            false_alarms,
                            gen_hashes,
                            false_alarm_hashes,
                            success_hashes,
                        )
                    else:
                        false_alarm_hashes += temp_hash_count
                        false_alarms = false_alarms + 1

            hash_of_password = apply_function(c, hash_of_password)
            gen_hashes += 1

    # no match is found after everything is complete
    return (
        False,
        false_alarms,
        gen_hashes,
        false_alarm_hashes,
        success_hashes,
    )

In [15]:
"""Full Hellman table method"""


def hellman_table(no_tables, no_chains, chain_len, no_iterations, hash_size):

    false_alarms = 0
    inverse_success = 0
    gen_hashes = 0
    false_alarm_hashes = 0
    success_hashes = 0
    global zeros
    global num_bytes
    global hash_data_type

    num_bytes = hash_size // 8
    zeros = 0
    zeros = zeros.to_bytes(num_bytes, "big")
    data_type_string = "S" + str(num_bytes)
    hash_data_type = np.dtype(data_type_string)

    # time Hellman table creation
    start = time.time()
    my_start_points = gen_start_points(no_tables, no_chains, hash_size)
    my_end_points = gen_end_points(my_start_points, chain_len)
    end = time.time()
    table_time = end - start

    my_passwords = generate_passwords(no_iterations, hash_size)

    # time search algorithm
    start = time.time()
    for i in range(no_iterations):
        x = search_chains(
            my_passwords[i][0],
            my_passwords[i][1],
            my_start_points,
            my_end_points,
            chain_len,
        )
        false_alarms += x[1]
        if x[0]:
            inverse_success += 1
        gen_hashes += x[2]
        false_alarm_hashes += x[3]
        success_hashes += x[4]

    end = time.time()
    search_time = end - start

    accuracy = inverse_success / no_iterations

    return (
        accuracy,
        false_alarms,
        table_time,
        search_time,
        gen_hashes,
        false_alarm_hashes,
        success_hashes,
    )

In [16]:
"""Master code"""

"""Format of output is as follows: no_tables,no_chains,chain_length,no_iterations,_hash_size,accuracy(percentage),no_collisions, table generation time,
search algorithm time, total execution time, coverage"""


def masterMethod(p):
    parameters = p
    start = time.time()
    my_results = hellman_table(
        no_tables=parameters[0],
        no_chains=parameters[1],
        chain_len=parameters[2],
        no_iterations=parameters[3],
        hash_size=parameters[4],
    )
    end = time.time()

    # accuracy
    parameters.append(my_results[0] * 100)

    # false alarms
    parameters.append(my_results[1])

    # general hashes
    parameters.append(my_results[4])

    # false alarm hashes
    parameters.append(my_results[5])

    # success hashes
    parameters.append(my_results[6])

    # table generation time
    parameters.append(my_results[2])

    # search algorithm time
    parameters.append(my_results[3])

    # total execution time
    total_time = end - start
    parameters.append(total_time)

    # write file
    writeResults(parameters)


parameters = [5, 337, 41, 100, 16]
masterMethod(parameters)

print(parameters)

[5, 337, 41, 100, 16, 26.0, 381, 15613, 5417, 597, 0.10634660720825195, 0.5434131622314453, 0.6497597694396973]


In [17]:
list_of_password_extractions

[['0xb69e', '0x4255', '0xb69e'],
 ['0x6db9', '0xa6a0', '0x6db9'],
 ['0x6bbf', '0xcf51', '0x6bbf'],
 ['0xee94', '0x2c53', '0xee94'],
 ['0xce3f', '0x45a1', '0xce3f'],
 ['0xd122', '0x2a9b', '0xd122'],
 ['0xa873', '0x1c9d', '0xa873'],
 ['0x5f5e', '0xc1fb', '0x5f5e'],
 ['0x3e85', '0xd64c', '0x3e85'],
 ['0x3f81', '0x518c', '0x3f81'],
 ['0xc0a2', '0x8e76', '0xc0a2']]

In [18]:
list_of_colliding_hashes

[['0x35ba', '0xccb7', '0x26b'],
 ['0xf225', '0x84b9', '0xe085'],
 ['0x4ac5', '0x93d7', '0xb8a7'],
 ['0xeaf0', '0x36a1', '0xed16'],
 ['0x74a7', '0x3290', '0x7ff6'],
 ['0x52b6', '0x90a', '0x92cf'],
 ['0x5e6e', '0x971f', '0xf2ea'],
 ['0xfe08', '0x3cbc', '0x7af6'],
 ['0xdf46', '0xd198', '0x10fc'],
 ['0x3345', '0x4e86', '0x9237'],
 ['0x9b8c', '0x8e76', '0xc0a2'],
 ['0xe409', '0xdd13', '0x15cd'],
 ['0x641', '0xb8a9', '0xeade'],
 ['0xe727', '0x8c3d', '0xa8ec'],
 ['0xd497', '0x3502', '0x1803']]

In [19]:
binary_string = list_of_colliding_hashes[1][1]